# Introduction to NLP fundamentals in Tensorflow

NLP has the goal of deriving information out of natural langugae (could be sequences, text or speech).

Another common term for NLP problems is seuence to sequence problems (seq2seq).


## Check for GPU

In [1]:
# !nvidia-smi -L

## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-11-22 01:18:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-22 01:18:40 (84.9 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as disaster or not disaster).

Dataset: https://www.kaggle.com/c/nlp-getting-started/data

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-11-22 01:18:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.203.128, 172.253.123.128, 142.250.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-11-22 01:18:43 (123 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in. One way to do so would be to use python: https://realpython.com/read-write-files-python/

Another way to do this is to use pandas.

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [6]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df_shuffled["text"].iloc[3]

'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.'

In [8]:
# what does the dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
# how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [11]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target> 0 else "(not real disaster)")
  print(f"Text:{text}\n")
  print("----\n")

Target: 0 (not real disaster)
Text:It may seem like our fire has been a little burnt out....

----

Target: 0 (not real disaster)
Text:These fucking police can't touch me  these fuck niggas ain't fucking w me

----

Target: 0 (not real disaster)
Text:her loyalty mission involves her kicking a shitty nobleman to death???? I love this elven weirdo

----

Target: 0 (not real disaster)
Text:*screams internally*

----

Target: 1 (real disaster)
Text:Turkish troops killed in Kurdish militant 'suicide attack' http://t.co/7cIbxls55f

----



### Split data into training and validation sets

In [12]:
from sklearn.model_selection import train_test_split


In [13]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [14]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

Wheb dealing with a text problem, one of the first thing we have to do is convert text to numbers.

There are a few ways to do this, namely:  
* **Tokenization** - direct mapping of a token (a token could be a word or a character) to number (text vectorization).
* **Embedding** - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned).

### Text vectorization (Tokenization)

In [16]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# # Use the default text vectorization parameters
# text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
#                                     standardize="lower_and_strip_punctuation",
#                                     split="whitespace",
#                                     ngrams=None, # creates groups of n words
#                                     output_mode="int",
#                                     output_sequence_length=None, # how long do you want your sequences to be?
#                                     pad_to_max_tokens=True)


In [18]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [19]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [20]:
len(train_sentences[0].split())

7

In [21]:
# find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

Average number of tokens (words) in a tweet is 15.

In [22]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15  # max length our sequences will be (e.g how many words from a tweet our model will see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)


In [23]:
# fit the text vectorizer to training data
text_vectorizer.adapt(train_sentences)

In [24]:
# create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [25]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 'I'm there!' Bride &amp; Groom on mountain cliff edge. Ha Ha just kidding. I WILL NOT EVER be there. Ha Ha - http://t.co/Io9ry1akON        

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  32,   74, 2534,   35, 3803,   11, 1118,  403, 5747, 1844, 1844,
          29, 5281,    8,   38]])>

In [26]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  # get all of the unique words in the our training data
top_5_words = words_in_vocab[:5] # get the most common words 
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f'5 most common words: {top_5_words}')
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


[UNK] - unknown token - stands for unknown (word out of the vocabulary)

### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use Tensorflow's embedding layer.

The parameters we care most about for our embedding layer:    
* `input_dim` = the size of our vocabulary
* ` output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [27]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length)

embedding

In [28]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# embed the random sentence (turn positive indexes into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Schools in Western Uganda still Burning down Buildings during Strikes....Strikes in Western Uganda always Lit literally..        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00994015,  0.00577831, -0.00043528, ...,  0.04774579,
          0.01212071, -0.0034212 ],
        [ 0.00211239, -0.03780458,  0.04884586, ...,  0.03071113,
          0.03307253,  0.00030943],
        [-0.00271506,  0.04528258, -0.04335169, ..., -0.01416766,
         -0.01951832, -0.04870909],
        ...,
        [ 0.04028371,  0.03936007,  0.03438186, ..., -0.03889229,
         -0.01814235,  0.02407361],
        [-0.00980448, -0.00040821,  0.00125802, ...,  0.04625118,
          0.04932446,  0.01113433],
        [-0.04881958,  0.00945355,  0.04712505, ...,  0.01100018,
         -0.02779353,  0.03544778]]], dtype=float32)>

In [29]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence.split()[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 9.94014740e-03,  5.77831268e-03, -4.35281545e-04,  1.05892643e-02,
         1.24777928e-02, -1.79928541e-02,  1.58855952e-02,  7.69805908e-03,
        -1.29309520e-02,  4.85724546e-02, -3.89259681e-02,  1.73329003e-02,
         2.91436911e-03,  1.55113824e-02,  2.26318836e-03,  4.05368954e-03,
        -7.92622566e-03,  3.39297093e-02,  1.21985301e-02,  4.95840646e-02,
        -4.64603417e-02, -1.60421506e-02,  4.12191488e-02, -3.44217941e-03,
        -2.41199732e-02, -3.00542470e-02,  7.89222866e-03, -4.04802337e-02,
         5.02648205e-03, -3.03645022e-02, -4.00644056e-02,  1.77922510e-02,
         2.52030604e-02, -2.67759208e-02, -2.18204390e-02, -3.08597926e-02,
         2.53888480e-02, -4.47162278e-02, -3.45362313e-02, -4.90672700e-02,
        -3.81481051e-02, -4.34619784e-02, -1.70738809e-02,  3.93620394e-02,
         4.01688330e-02, -1.12074837e-02,  6.40075281e-03,  7.45686144e-03,
        -1.29419789e-02,  1.39381997e-0

## Modelling a text dataset

Now we've got a way to turn our tetxt sequences into numbers, it's time to start building a series of modelling experiments.

Let;s start with a baseline and move on from there.

 * Model 0: Naive Bayes (baseline) 
 * Model 1: Feed-forward neural network
 * Model 2: LSTM model (RNN)
 * Model 3: GRU model (RNN)
 * Model 4: Bidirectional-LSTM model (RNN)
 * Model 5: 1D Convolutional Neural network (CNN)
 * Model 6: Tensorflow Hub pretrained feature extractor (using transfer learning for NLP)
 * Model 7: Same as model 6 with 10% of training data

We will use the standard steps in modelling with tensorflow:    
* Create a model
* Build a model
* Fit the model
* Evaluate the model






### Model 0: Getting a Baseline Model

As with all machine elarning modelling experiments, it's important to create a baseline model so you've gto a benchmark for future experiments to build upon.

To create our baseline, we'll sue Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.


In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of :{baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of :79.27%


In [32]:
train_df.target.value_counts() # slightly imbalanced classes

0    4342
1    3271
Name: target, dtype: int64

In [33]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating an evaluation function for our model experiments

Let's create a function to compare our model's predictions with the truth label using the following metrics:   
* Accuracy
* Precision
* Recall
* F1-score

In [34]:
# Function to evaluate: accuracy , precision, recall , f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  '''
  Calculates model accuracy, precision, recall and f1-score of a binary classification model.
  '''
  # calculat model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100
  # calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results 

In [35]:
# get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [36]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"

# Build the model with functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)  # create an embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x)  # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x)  # create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20211122-011845
Epoch 1/5
215/215 [==============================] - 4s 16ms/step - loss: 0.6117 - accuracy: 0.6866 - val_loss: 0.5376 - val_accuracy: 0.7493
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4428 - accuracy: 0.8145 - val_loss: 0.4725 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3479 - accuracy: 0.8594 - val_loss: 0.4609 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2861 - accuracy: 0.8910 - val_loss: 0.4622 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2387 - accuracy: 0.9123 - val_loss: 0.4802 - val_accuracy: 0.7900


In [39]:
# check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4802 - accuracy: 0.7900


[0.4801863133907318, 0.7900262475013733]

In [40]:
#make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [41]:
# look at a single prediction
model_1_pred_probs[0]

array([0.335145], dtype=float32)

In [42]:
# Look  at the first 10 predictions
model_1_pred_probs[:10]

array([[0.335145  ],
       [0.83417773],
       [0.9980501 ],
       [0.08792666],
       [0.10919306],
       [0.9329672 ],
       [0.9141898 ],
       [0.99379194],
       [0.96276736],
       [0.22897881]], dtype=float32)

In [43]:
# convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [44]:
# calculate model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 79.00262467191601,
 'f1': 0.7866259949990327,
 'precision': 0.7964718621790533,
 'recall': 0.7900262467191601}

In [45]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [46]:
import numpy as np
np.array(list(model_1_results.values())) >  np.array(list(baseline_results.values()))

array([False, False, False,  True])

Comparing results shows that baseline model outperforms our first deep leanring based dense model.


## Visualizing learned embeddings

In [47]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

In [48]:
len(words_in_vocab)

10000

In [49]:
# first 10 most common words
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [50]:
max_vocab_length

10000

In [51]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [52]:
# get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[ 0.01823394, -0.06905591,  0.00959345, ...,  0.01379115,
        -0.02526248,  0.01233402],
       [-0.009679  , -0.04669121, -0.02075852, ...,  0.01120809,
         0.00658702,  0.01890875],
       [ 0.02131985,  0.00671595,  0.02653384, ...,  0.04322812,
         0.02514545,  0.02300458],
       ...,
       [ 0.04706607, -0.0005859 , -0.04014157, ..., -0.02090134,
        -0.01535139, -0.00048393],
       [ 0.03409407, -0.03720106, -0.01192651, ...,  0.06868954,
         0.009566  ,  0.03031763],
       [ 0.05853991, -0.03419592, -0.02397197, ...,  0.07778879,
         0.04682536,  0.114053  ]], dtype=float32)

In [53]:
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it:    

To do so, Tensorflow has a handy tool called **Projector**: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on **word embeddings** themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [54]:
# # create embedding files (we've got this from Tensorflow's word embeddings documentation)
# import io
# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [55]:
# # Download files from colab to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks (RNN's)

RNNs are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

- MIT deep learning: http://introtodeeplearning.com/
- Andrej Karpathy's blog post: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2 - LSTM

LSTM stands for **Long short term memory** (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:    
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/Dense) -> Output (label probability)
``` 

In [58]:
# Create an LSTM model

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences=True)(x) # when you're stacking RNN cells together, you need to return_sequences = true  
print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM") 

(None, 15, 128)
(None, 15, 64)


In [59]:
# Get a summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                      

In [61]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [63]:
# fit the model
model_2_history = model_2.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20211122-012916
Epoch 1/5
215/215 [==============================] - 12s 41ms/step - loss: 0.2186 - accuracy: 0.9244 - val_loss: 0.5665 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 8s 36ms/step - loss: 0.1622 - accuracy: 0.9409 - val_loss: 0.7164 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 8s 37ms/step - loss: 0.1327 - accuracy: 0.9494 - val_loss: 0.6939 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 8s 37ms/step - loss: 0.1052 - accuracy: 0.9577 - val_loss: 0.9736 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0824 - accuracy: 0.9664 - val_loss: 1.2700 - val_accuracy: 0.7717


In [64]:
# Make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

array([[3.0907989e-04],
       [4.4239292e-01],
       [9.9991542e-01],
       [3.2060742e-03],
       [1.1440077e-04],
       [9.9985641e-01],
       [9.4922984e-01],
       [9.9993193e-01],
       [9.9992597e-01],
       [6.3492900e-01]], dtype=float32)

In [65]:
# Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [66]:
# Calculate model 2 results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)

In [67]:
model_2_results

{'accuracy': 77.16535433070865,
 'f1': 0.7693184984034235,
 'precision': 0.7738402637184817,
 'recall': 0.7716535433070866}

In [68]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 3: GRU

Another popular and effective RNN component is the **GRU** or **gated recurrent unit**.

The GRU cell has similar features to an LSTM cell but has less parameters.

Resource: https://towardsdatascience.com/understanding-gru-networks-2ef37df6c9be

In [79]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x) # if you want to stack RNN layers on top of each other, you need return_sequences=True
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [80]:
# get a summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_10 (GRU)                (None, 64)                37248     
                                                                 
 dense_18 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [81]:
# compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [82]:
# fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20211122-015729
Epoch 1/5
215/215 [==============================] - 9s 31ms/step - loss: 0.1599 - accuracy: 0.9365 - val_loss: 0.7541 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0865 - accuracy: 0.9675 - val_loss: 0.7837 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0704 - accuracy: 0.9720 - val_loss: 0.7928 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0616 - accuracy: 0.9730 - val_loss: 1.2863 - val_accuracy: 0.7651
Epoch 5/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0513 - accuracy: 0.9778 - val_loss: 1.2607 - val_accuracy: 0.7677


In [83]:
# make some predictions with our gru model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[9.7517753e-01],
       [7.2118533e-01],
       [9.9988776e-01],
       [2.3518619e-01],
       [7.0693728e-05],
       [9.9969041e-01],
       [5.8729303e-01],
       [9.9994457e-01],
       [9.9990779e-01],
       [8.5242534e-01]], dtype=float32)

In [84]:
# convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [85]:
# calculate model 3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 76.77165354330708,
 'f1': 0.7662770891654436,
 'precision': 0.7681410880728078,
 'recall': 0.7677165354330708}

In [86]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 4: Bidirectional RNN

Normal RNN's go from left to right however, bidirectional RNN goes from right to left as well as left to right.

In [90]:
# build a bidirectional RNN
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [91]:
# Get a summary
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [92]:
# compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [93]:
# fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20211122-021806
Epoch 1/5
215/215 [==============================] - 12s 41ms/step - loss: 0.1032 - accuracy: 0.9702 - val_loss: 1.0089 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0514 - accuracy: 0.9784 - val_loss: 1.2971 - val_accuracy: 0.7703
Epoch 3/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0454 - accuracy: 0.9784 - val_loss: 1.4176 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0434 - accuracy: 0.9806 - val_loss: 1.2670 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 8s 36ms/step - loss: 0.0397 - accuracy: 0.9816 - val_loss: 1.4282 - val_accuracy: 0.7703


In [94]:
# make predictions with bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[1.3440549e-03],
       [7.9761732e-01],
       [9.9997306e-01],
       [2.5411865e-01],
       [1.3346498e-05],
       [9.9910545e-01],
       [6.7561316e-01],
       [9.9998522e-01],
       [9.9997926e-01],
       [9.9062079e-01]], dtype=float32)

In [95]:
# convert pred probs to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [96]:
# calculate the results of bidirectional model
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.03412073490814,
 'f1': 0.7684486602580174,
 'precision': 0.7715893693867238,
 'recall': 0.7703412073490814}

## Convolutional Neural Networks for Text ( and other types of sequences)

CNNs are typically used for images but images are 2D (height*width)..however text data is 1D.

Conv2D is used for image data but Conv1D can be used for text.

The typical structure of a Conv1D model fr sequences:    

```
Input (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + Pooling) -> Ouputs (class probabilities)
```

### Model 5: Conv1D

For different explanations of parameters, see:    
* https://poloclub.github.io/cnn-explainer/
* Difference between same and valid padding: stackoverflow

In [106]:
# Test out our embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=64,
                        kernel_size=5, # this is also refereed to as  ngram of 5 (it looks at 5 words at a time)
                        activation="relu",
                        padding="valid") #(output is smaller than the input shape with "valid")
conv_1d_output = conv_1d(embedding_test) # pass test embedding through conv1d layer
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)  # equivalent to 'get the most important feature' or 'get the feature with the highest value'


embedding_test.shape, conv_1d_output.shape, max_pool_output.shape


(TensorShape([1, 15, 128]), TensorShape([1, 11, 64]), TensorShape([1, 64]))

In [105]:
# embedding_test

In [104]:
# conv_1d_output

In [103]:
# max_pool_output